In [ ]:
import torch
import transformers
from transformers import pipeline
from transformers import DistilBertModel, DistilBertForSequenceClassification, DistilBertTokenizer
from transformers.models.distilbert import modeling_distilbert
import datasets
from transformers import TrainingArguments
import numpy as np
import evaluate

/home/simon/coding/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-10 16:24:58.819062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741620298.883636   16460 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741620298.901421   16460 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 16:24:59.033660: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical ope

In [ ]:
class iMultiHeadSelfAttention(nn.Module):
    def __init__(self, config: PretrainedConfig):
        super().__init__()
        self.config = config

        self.n_heads = config.n_heads
        self.dim = config.dim
        self.dropout = nn.Dropout(p=config.attention_dropout)
        self.is_causal = False

        # Have an even number of multi heads that divide the dimensions
        if self.dim % self.n_heads != 0:
            # Raise value errors for even multi-head attention nodes
            raise ValueError(f"self.n_heads: {self.n_heads} must divide self.dim: {self.dim} evenly")

        self.q_lin = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.k_lin = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.v_lin = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.out_lin = nn.Linear(in_features=config.dim, out_features=config.dim)

        self.gamma = 1
        self.delta = 1
        self.eta = 1

        self.pruned_heads: Set[int] = set()
        self.attention_head_size = self.dim // self.n_heads

    def prune_heads(self, heads: List[int]):
        if len(heads) == 0:
            return
        heads, index = find_pruneable_heads_and_indices(
            heads, self.n_heads, self.attention_head_size, self.pruned_heads
        )
        # Prune linear layers
        self.q_lin = prune_linear_layer(self.q_lin, index)
        self.k_lin = prune_linear_layer(self.k_lin, index)
        self.v_lin = prune_linear_layer(self.v_lin, index)
        self.out_lin = prune_linear_layer(self.out_lin, index, dim=1)
        # Update hyper params
        self.n_heads = self.n_heads - len(heads)
        self.dim = self.attention_head_size * self.n_heads
        self.pruned_heads = self.pruned_heads.union(heads)

    def compute_relu_attention(self, scores: torch.Tensor, mask: torch.Tensor, 
                               head_mask: Optional[torch.Tensor], v: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        scores = F.relu(scores)
        weights = scores
        M = torch.max(torch.maximum(v, -v)) + 1
        mask = self.dropout(mask.to(scores.dtype))
        mask = (mask == 0).unsqueeze(1).unsqueeze(2)
        scores = scores.masked_fill(mask, M)
        v_t = v.transpose(-1, -2)
        
        if head_mask is not None:
            weights = weights * head_mask
        
        context = self.compute_signed_inhibitor(scores, v, v_t)

        return context, weights
    
    def compute_signed_inhibitor(self, scores: torch.Tensor, v: torch.Tensor, v_t: torch.Tensor) -> torch.Tensor:
        pos_v = F.relu(v_t)
        neg_v = -F.relu(-v_t)
        v_sum = torch.sum(v, dim=-2, keepdim=True)
        dist1 = torch.cdist(scores, pos_v, p=1)
        dist2 = torch.cdist(scores, -neg_v, p=1)
        return 0.5 * (v_sum + dist1 - dist2)

    def forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        mask: torch.Tensor,
        head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, ...]:
        """
        Parameters:
            query: torch.tensor(bs, seq_length, dim)
            key: torch.tensor(bs, seq_length, dim)
            value: torch.tensor(bs, seq_length, dim)
            mask: torch.tensor(bs, seq_length)

        Returns:
            weights: torch.tensor(bs, n_heads, seq_length, seq_length) Attention weights context: torch.tensor(bs,
            seq_length, dim) Contextualized layer. Optional: only if `output_attentions=True`
        """
        bs, q_length, dim = query.size()
        k_length = key.size(1)
        # assert dim == self.dim, f'Dimensions do not match: {dim} input vs {self.dim} configured'
        # assert key.size() == value.size()

        dim_per_head = self.dim // self.n_heads

        mask_reshp = (bs, 1, 1, k_length)

        def shape(x: torch.Tensor) -> torch.Tensor:
            """separate heads"""
            return x.view(bs, -1, self.n_heads, dim_per_head).transpose(1, 2)

        def unshape(x: torch.Tensor) -> torch.Tensor:
            """group heads"""
            return x.transpose(1, 2).contiguous().view(bs, -1, self.n_heads * dim_per_head)

        q = shape(self.q_lin(query))  # (bs, n_heads, q_length, dim_per_head)
        k = shape(self.k_lin(key))  # (bs, n_heads, k_length, dim_per_head)
        v = shape(self.v_lin(value))  # (bs, n_heads, k_length, dim_per_head)

        #q = q / math.sqrt(dim_per_head)  # (bs, n_heads, q_length, dim_per_head)
        #scores = torch.matmul(q, k.transpose(2, 3))  # (bs, n_heads, q_length, k_length)
        scale = self.gamma/math.sqrt(dim_per_head)
        scores = torch.cdist(q, k, p=1) * scale # Z

        mean = torch.mean(scores, dim = -1, keepdim = True)
        scores -= mean
            
        #shift score
        if self.alpha > 0:
            scores += self.alpha  

        context, weights = self.compute_relu_attention(scores, mask, head_mask, v)
        context *= self.beta #scale context

        context = unshape(context)  # (bs, q_length, dim)
        context = self.out_lin(context)  # (bs, q_length, dim)

        if output_attentions:
            return (context, weights)
        else:
            return (context,)

In [ ]:
modeling_distilbert.MultiHeadSelfAttention = iMultiHeadSelfAttention

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained()
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
p = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer)
p("I like eating [MASK]")